In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Dropout, MaxPool1D
from keras.layers.convolutional import Conv1D
from keras.optimizers import Adam
import os

In [115]:
data = pd.read_csv("training.csv")
data.drop(['Unnamed: 0'], axis = 1, inplace=True)
train_df = data.iloc[:1000]
test_df = data.iloc[1000:]

In [117]:
train_y = train_df.pop("TSLA")
train_y = train_df.pop("Volume")
train_x = train_df

test_y = test_df.pop("TSLA")
test_y = test_df.pop("Volume")
test_x = test_df

In [118]:
train_x.shape

(1000, 5)

In [119]:
train_x_arr = np.reshape(train_x.values, train_x.values.shape + (1,))
train_y_arr = np.reshape(train_y.values, train_y.values.shape + (1,1,))
test_x_arr = np.reshape(test_x.values, test_x.values.shape + (1,))
test_y_arr = np.reshape(test_y.values, test_y.values.shape + (1,1,))

In [121]:
train_x_arr.shape, train_y_arr.shape

((1000, 5, 1), (1000, 1, 1))

In [136]:
model = Sequential()
model.add(LSTM(1, input_shape=(5,1), return_sequences=True, dtype='float32'))
model.add(LSTM(1, return_sequences=True, dtype='float32'))
model.add(MaxPool1D(pool_size=5))

In [137]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_45 (LSTM)               (None, 5, 1)              12        
_________________________________________________________________
lstm_46 (LSTM)               (None, 5, 1)              12        
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 1, 1)              0         
Total params: 24
Trainable params: 24
Non-trainable params: 0
_________________________________________________________________


In [125]:
optimizer = Adam(lr=0.1)
model.compile(optimizer=optimizer,
             loss='mean_absolute_percentage_error',
             metrics=['accuracy'])

In [126]:
model.fit(train_x_arr, train_y_arr, epochs=20, batch_size=20, 
          verbose=1)

Epoch 1/20
1000/1000 [==============================] - 4s - loss: 99.9990 - acc: 0.0000e+00        
Epoch 2/20
1000/1000 [============

In [53]:
trainScore = model.evaluate(train_x[:,1:], train_y[:,1:], verbose=1)

 32/330 [=>............................] - ETA: 0s

In [59]:
testScore = model.evaluate(test_x[:,1:], test_y[:,1:], verbose=1)

 32/140 [=====>........................] - ETA: 0s

In [127]:
added = np.vstack([train_x_arr, test_x_arr])

In [128]:
added.shape

(1258, 5, 1)

In [129]:
test_pred = model.predict(added)

In [130]:
test_pred[1000:]

array([[[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404053]],

       [[ 1005.20404

In [110]:
import pylab as pl
time = np.arange(0, 258, 1)
actual = test_y.values
pred = train_x.values

In [111]:
pred

array([[  7.08000000e+00,   1.39000000e+00,   1.93200000e+01,
          7.29000000e+00,   1.15796900e+06,   0.00000000e+00],
       [  7.33000000e+00,   1.21000000e+00,   1.92500000e+01,
          7.15000000e+00,   3.09718000e+06,   3.20000000e+01],
       [  6.96000000e+00,   1.37000000e+00,   1.89100000e+01,
          7.29000000e+00,   1.92812300e+06,   3.30000000e+01],
       ..., 
       [  6.42000000e+00,   6.99900000e-01,   3.06800000e+01,
          9.45000000e+00,   3.66508800e+06,   0.00000000e+00],
       [  6.42000000e+00,   6.50000000e-01,   3.09000000e+01,
          9.36500000e+00,   3.21777900e+06,   0.00000000e+00],
       [  6.36000000e+00,   7.00000000e-01,   3.09300000e+01,
          9.34000000e+00,   3.19810500e+06,   0.00000000e+00]])

In [68]:
look_back = 10 # number of previous time steps to use as input to predict the next time period
trainPredict = model.predict(train_x[:,1:])
testPredict = model.predict(test_x[:,1:])

# shift train predictions for plotting
trainPredictPlot = np.empty_like(data[:,1:])
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
 
# shift test predictions for plotting
testPredictPlot = np.empty_like(data[:,1:])
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(data[:,1:])-1, :] = testPredict

ValueError: could not broadcast input array from shape (140,1) into shape (118,3)